# 2. Intro to Catalyst

This tutorial has been adapted from the [Catalyst docs](https://docs.sciml.ai/Catalyst/stable/network_analysis/crn_theory/) (see also [here](https://docs.sciml.ai/Catalyst/stable/network_analysis/odes/)).

In [ ]:
using Pkg
Pkg.activate(".")

In [ ]:
using Catalyst
using LinearAlgebra
using Latexify

## Constructing CRN
Chemical reaction networks are created using the `@reaction_network` macro. Each line represents a reaction: the first entry is the rate parameter, and the second entry is the reaction stoichiometry.

The following is an example network from Shinar and Feinberg (2010).

In [ ]:
rn = @reaction_network begin
    α, A + B --> 2B
    β, B --> A
end

Note: use backslash `\` to type unicode characters e.g. `\beta<tab>` -> β

In [ ]:
# Accessing network variables
@show species(rn)
@show parameters(rn)
@show reactions(rn)

# Basic properties
@show numspecies(rn)
@show numparams(rn)
@show numreactions(rn)

In [ ]:
[rn.A, rn.B]

In [ ]:
conservedequations(rn)

In [ ]:
N = netstoichmat(rn) 

In [ ]:
deficiency(rn)

## Example 2
We can calculate several relevant properties in chemical reaction network theory.

In [ ]:
rn = @reaction_network begin
    (k1,k2), A + B <--> C
    k3, C --> D+E
    (k4,k5), D+E <--> F
    (k6,k7), 2A <--> B+G
    k8, B+G --> H
    k9, H --> 2A
end

In [ ]:
conservationlaws(rn)

In [ ]:
conservedequations(rn)

In [ ]:
s = rank(netstoichmat(rn))

In [ ]:
nspecies = numspecies(rn)
nlaws = length(conservedequations(rn))

nspecies - nlaws == s

In [ ]:
subnets = subnetworks(rn)
reactions.(subnets)

### Plotting networks

In [ ]:
# additional packages to plot CRN
Pkg.add("CairoMakie")
Pkg.add("GraphMakie")
Pkg.add("NetworkLayout")

In [ ]:
import CairoMakie, GraphMakie, NetworkLayout

In [ ]:
plot_complexes(subnets[1])

In [ ]:
plot_complexes(subnets[2])

### Deficiency
Recall the definition of reaction network deficiency $\delta$
$$
\delta = n - l - s
$$
where $n$ is the \# complexes, $l$ is the \# linkage classes, and $s$ is the rank of the stochiometry matrix.

In [ ]:
δ = deficiency(rn)

In [ ]:
n = length(complexes)
l = length(linkageclasses(rn))
δ == n - l - s

In [ ]:
# Reversible system
rev = @reaction_network begin
  (k1,k2),A <--> B
  (k3,k4),A + C <--> D
  (k5,k6),D <--> B+E
  (k7,k8),B+E <--> A+C
end
@show isreversible(rev)

# Weakly reversible system
wrev = @reaction_network begin
  (k1,k2),A <--> B
  k3, A + C --> D
  k4, D --> B+E
  k5, B+E --> A+C
end
@show isreversible(wrev)
subnets = subnetworks(wrev)
@show isweaklyreversible(wrev, subnets)

In [ ]:
rxs = reactions(rn)
for rx in rxs
    @show rx
    @show ismassaction(rx, rn)
end

In [ ]:
all(rx -> ismassaction(rx, rn), reactions(rn))

To satisfy the conditions for deficiency zero theorems, we require reaction networks to be *mass action* and have *deficiency zero*. If such a network is *weakly reversible*, then the ODEs will have exactly one equilibrium solution within each positive stoichiometric compatibility class. Otherwise, the ODEs do not have a valid solution

In [ ]:
def0_rn = @reaction_network begin
  (k1,k2),A <--> 2B
  (k3,k4), A + C <--> D
  k5, B+E --> C + D
end
reactioncomplexes(def0_rn)
subnets = subnetworks(def0_rn)
isma = all(rx -> ismassaction(rx,def0_rn), reactions(def0_rn))
def = deficiency(def0_rn)
iswr = isweaklyreversible(def0_rn, subnets)

@show isma
@show def
@show iswr

satisfiesdeficiencyzero(def0_rn)

In [ ]:
satisfiesdeficiencyzero(wrev)

In [ ]:
# Deficiency One
def1_network = @reaction_network begin
    (k1, k2), A <--> 2B
    k3, C --> 2D
    k4, 2D --> C + E
    (k5, k6), C + E <--> E + 2D
end
plot_complexes(def1_network);

In [ ]:
@assert satisfiesdeficiencyone(def1_network)
deficiency(def1_network)

## Reaction Network ODEs

There are multiple ways to represent a reaction network as ODEs. See the [docs](https://docs.sciml.ai/Catalyst/stable/network_analysis/odes/).

### Stoichiometric form
$$
\frac{d\mathbf{x}}{dt} = N\mathbf{v}(\mathbf{x})
$$
where $\mathbf{x}(t)$ is the vector of species, $N$ is the stoichiometric matrix, and $\mathbf{v}(\mathbf{x})$ is the rate law vector.

In [ ]:
rn = @reaction_network begin
    k, 2A + 3B --> A + 2C + D
    b, C + D --> 2A + 3B
end

In [ ]:
N = netstoichmat(rn)

In [ ]:
v = oderatelaw.(reactions(rn), combinatoric_ratelaw=false)

In [ ]:
# NB oderatelaw may not be mass action
rx = @reaction hillr(X, α, K, n), 0 --> Y
oderatelaw(rx)

NB: if `combinatoric_ratelaw=true` then a scaling factor of $1/k!$ is applied where $k$ is the coefficient.

In [ ]:
# NB oderatelaw may not be mass action
rx = @reaction β, 3X --> Y
oderatelaw(rx)

In [ ]:
N * v

In [ ]:
odesys = convert(ODESystem, rn, combinatoric_ratelaws=false)

### Complex representation
$$
\frac{d\mathbf{x}}{dt} = ZB\mathbf{v}(\mathbf{x})
$$

In [ ]:
rn

In [ ]:
x = species(rn)

In [ ]:
Z = complexstoichmat(rn)

In [ ]:
# NB: Z' is the transpose of Z
Z' * x

In [ ]:
B = incidencemat(rn)

In [ ]:
N == Z*B

The reaction rate vector can also be defined as
$$
\mathbf{v}(\mathbf{x}) = K\Phi(\mathbf{x})
$$
where $K$ is the *flux matrix* and $\Phi(\mathbf{x})$ is the vector of *monomials* that define the mass action rate laws.

In [ ]:
K = fluxmat(rn)

In [ ]:
Φ = massactionvector(rn; combinatoric_ratelaws=false)

In [ ]:
K * Φ

Lastly, $A_k = BK$ is the negative of the weighted *Laplacian matrix* of the reaction network.

In [ ]:
A_k = laplacianmat(rn)

$$
\begin{align*}
    \frac{d\mathbf{x}}{dt} &= N\mathbf{v}\\
    &= NK\Phi\\
    &= ZBK\Phi\\
    &= ZA_k\Phi
\end{align*}
$$

## Other useful functions

See the [API docs](https://docs.sciml.ai/Catalyst/stable/api/network_analysis_api/) for more info.

In [ ]:
rn = @reaction_network begin
  (k1,k2),A <--> B
  k3, A + C --> D
  k4, D --> B+E
  k5, B+E --> A+C
end

In [ ]:
rates = Dict([:k1 => 2.4, :k2 => 4., :k3 => 10., :k4 => 5.5, :k5 => 0.4])
@show iscomplexbalanced(rn, rates)
@show isdetailedbalanced(rn, rates)

In [ ]:
rn

Substrate $S$ and product $P$ stoichiometry matrices. Note that $N = P - S$

In [ ]:
display(substoichmat(rn))
display(prodstoichmat(rn))
display(netstoichmat(rn))